In [1]:
import dash
import pandas as pd
import plotly.express as px
from dash import dcc, html, Input, Output

In [2]:
population = pd.read_csv("../Dash_crossfiltering/population.csv")
new_books = pd.read_csv("../Dash_crossfiltering/new-books-per-million.csv")

In [3]:
population.head(5)

,Entity,Code,Year,Population (historical estimates)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166


In [4]:
new_books.head(5)

,Entity,Code,Year,Book titles per capita (Fink-Jensen 2015)
0,Algeria,DZA,1953,10.596210
1,Algeria,DZA,1954,8.115622
2,Algeria,DZA,1963,12.596606
3,Algeria,DZA,1964,13.605609
4,Algeria,DZA,1965,10.950347


In [5]:
new_books.shape

(8312, 4)

In [6]:
new_books = new_books[(new_books["Year"]>=1940) & (new_books["Year"]<=1996)]

In [7]:
new_books.shape

(2852, 4)

In [8]:
selected_books = new_books[new_books["Year"]==1996][["Entity", "Book titles per capita (Fink-Jensen 2015)"]]

In [9]:
selected_books.shape

(48, 2)

In [10]:
selected_books.head(5)

,Entity,Book titles per capita (Fink-Jensen 2015)
17,Algeria,23.423261
67,Argentina,276.041813
237,Austria,985.472115
794,Belgium,1148.548561
830,Brazil,113.694110


In [11]:
selected_pops = population[population["Year"]==1996][["Entity", "Population (historical estimates)"]]

In [12]:
selected_pops.head(5)

,Entity,Population (historical estimates)
233,Afghanistan,18853444
492,Africa,735361106
751,Albania,3098699
1010,Algeria,29266415
1124,American Samoa,54209


In [13]:
data = pd.merge(selected_books, selected_pops, how="inner", on="Entity")

In [14]:
data.head(5)

,Entity,Book titles per capita (Fink-Jensen 2015),Population (historical estimates)
0,Algeria,23.423261,29266415
1,Argentina,276.041813,35246376
2,Austria,985.472115,8017852
3,Belgium,1148.548561,10208265
4,Brazil,113.694110,164614682


In [15]:
scatter_figure = px.scatter(data_frame=data,
                 x="Book titles per capita (Fink-Jensen 2015)",
                 y="Population (historical estimates)",
                 text="Entity",
                 log_x=True,
                 log_y=True,
                 template="none",
                 title="Country population vs number of new book titles per capita in 1996")
scatter_figure.update_traces(textposition="bottom right")
scatter_figure.show()

In [16]:
# Initialize app
app = dash.Dash()

# app layout
app.layout = html.Div(children=[
        # Scatter plot section
        dcc.Graph(
            id="scatter",
            figure=scatter_figure,
        ),
        # Placeholder for detailed graph
        html.Div(id="detailed-graph", style={"margin-top": "20px"}), 
    ],
    style={
        "fontFamily": "Verdana",
        "padding": "20px",
        "background-color": "#f7f7f7",
        "color": "#444",
    }
)

# Callback to update the detailed graph based on click
@app.callback(
    Output("detailed-graph", "children"),
    [Input("scatter", "clickData")]
)
def display_detailed_graph(click_data):
    """
    Display a detailed line chart below the scatter plot when a point is clicked.
    Args:
        click_data: Information about the clicked data point.
    Returns:
        A new line chart as a Dash component.
    """
    if click_data is None:
        return None

    clicked_point = click_data["points"][0]
    selected_data = new_books[new_books["Entity"] == clicked_point["text"]]

    # Create a line chart for the selected country
    fig = px.line(
        data_frame=selected_data,
        x="Year",
        y="Book titles per capita (Fink-Jensen 2015)",
        template="none",
        title=f"Book titles per capita in {clicked_point['text']} (1940-1996)"
    )

    # Return the graph as a child of the container
    return dcc.Graph(figure=fig, style={"height": "400px", "width": "100%"})

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)